In [1]:
pip install pandas numpy pillow matplotlib scikit-learn ace_tools

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, svm

df = pd.read_csv("FAOSTAT_data_en_4-24-2025.csv")
df2 = pd.read_csv("CropData.csv")

In [9]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, svm

df1 = pd.read_csv("Total_YoY_Yield_Loss_Clean.csv")
df2 = pd.read_csv("RainfallByYear.csv")
df3 = pd.read_csv("australia_temperature_trends.csv")
df3

,Year,Month,Monthly_Anomaly,Monthly_Uncertainty,Annual_Anomaly,Annual_Uncertainty,FiveYear_Anomaly,FiveYear_Uncertainty,TenYear_Anomaly,TenYear_Uncertainty,TwentyYear_Anomaly,TwentyYear_Uncertainty
0,1876,2,-0.410,1.455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1876,3,0.103,1.504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1876,4,-0.749,1.114,-0.603,0.492,NaN,NaN,NaN,NaN,NaN,NaN
3,1876,5,-0.174,0.945,-0.556,0.461,NaN,NaN,NaN,NaN,NaN,NaN
4,1876,6,-0.978,0.950,-0.435,0.467,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1734,2020,8,1.439,0.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1735,2020,9,2.590,0.222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1736,2020,10,0.985,0.148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1737,2020,11,2.404,0.173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load datasets
yield_loss_df = pd.read_csv('Total_YoY_Yield_Loss_Clean.csv')
rainfall_df = pd.read_csv('RainfallByYear.csv')
temp_df = pd.read_csv('australia_temperature_trends.csv')

# Prepare yield loss DataFrame
yield_loss_df['YearNumeric'] = yield_loss_df['Year'].str[:4].astype(int)
yield_loss_df = yield_loss_df[['YearNumeric', 'Yield Loss YoY']].rename(
    columns={'Yield Loss YoY': 'YieldLoss'}
)

# Prepare rainfall DataFrame
rainfall_df = rainfall_df.rename(columns={'Rainfall_mm': 'Rainfall'})

# Prepare temperature DataFrame: aggregate annual anomaly
temp_annual = (
    temp_df
    .groupby('Year', as_index=False)['Annual_Anomaly']
    .mean()
    .rename(columns={'Annual_Anomaly': 'TemperatureAnomaly'})
)

# Merge datasets on year
df = (
    yield_loss_df
    .merge(rainfall_df, left_on='YearNumeric', right_on='Year')
    .merge(temp_annual, left_on='YearNumeric', right_on='Year')
)
df = df[['YearNumeric', 'Rainfall', 'TemperatureAnomaly', 'YieldLoss']]

# Fit a simple linear regression model
X = df[['Rainfall', 'TemperatureAnomaly']]
y = df['YieldLoss']
model = LinearRegression()
model.fit(X, y)

# Create properly-named sample DataFrame for 2025
rainfall_dummy = rainfall_df.sort_values('Year').tail(3)['Rainfall'].mean()
temp_dummy = temp_annual.sort_values('Year').tail(3)['TemperatureAnomaly'].mean()

sample_2025 = pd.DataFrame({
    'Rainfall': [rainfall_dummy],
    'TemperatureAnomaly': [temp_dummy]
})
pred_2025 = model.predict(sample_2025)[0]

# Prepare display DataFrame with fitted values
df_display = df.copy()
df_display['PredictedYieldLoss'] = model.predict(X)

# Append the 2025 prediction via pd.concat, using np.nan instead of None
new_row = pd.DataFrame([{
    'YearNumeric': 2025,
    'Rainfall': rainfall_dummy,
    'TemperatureAnomaly': temp_dummy,
    'YieldLoss': np.nan,
    'PredictedYieldLoss': pred_2025
}])
df_display = pd.concat([df_display, new_row], ignore_index=True)

df_display

,YearNumeric,Rainfall,TemperatureAnomaly,YieldLoss,PredictedYieldLoss
0,1989,480.300000,0.217583,0.0,5304.602980
1,1990,413.170000,0.580417,0.0,6178.616833
2,1991,460.810000,0.800917,2288.9,4172.296064
3,1992,447.910000,0.345833,0.0,5862.066679
4,1993,487.960000,0.381917,0.0,4608.288658
5,1994,338.680000,0.237083,12872.5,9311.525729
6,1995,518.150000,0.371750,0.0,3771.535276
7,1996,462.260000,0.613167,0.0,4675.122173
8,1997,514.170000,0.518833,4276.4,3459.225558
9,1998,548.860000,0.949917,0.0,1213.834109
